In [8]:
import pandas as pd
import re

human_complexes = pd.read_csv("human_complex_portal.tsv", sep='\t')

In [9]:
human_complexes

,#Complex ac,Recommended name,Aliases for complex,Taxonomy identifier,Identifiers (and stoichiometry) of molecules in complex,Evidence Code,Experimental evidence,Go Annotations,Cross references,Description,Complex properties,Complex assembly,Ligand,Disease,Agonist,Antagonist,Comment,Source,Expanded participant list
0,CPX-1,SMAD2-SMAD3-SMAD4 complex,SMAD2/SMAD3/SMAD4 transcription factor complex,9606,P84022(1)|Q13485(1)|Q15796(1),ECO:0005547(biological system reconstruction e...,-,GO:0071144(heteromeric SMAD protein complex)|G...,reactome:R-HSA-9736938(identity)|reactome:R-HS...,A transcription factor complex which binds to ...,Preferential formation of the regulatory R-Sma...,Heterotrimer,-,-,-,-,-,"psi-mi:""MI:0469""(IntAct)",P84022(1)|Q13485(1)|Q15796(1)
1,CPX-8,"bZIP transcription factor complex, ATF4-CREB1",-,9606,P16220(1)|P18848(1),ECO:0005544(biological system reconstruction e...,-,GO:0000977(RNA polymerase II transcription reg...,complex portal:CPX-8(complex-primary)|complex ...,Transcription factor complex that binds the cA...,-,Heterodimer,-,-,-,-,-,"psi-mi:""MI:0469""(IntAct)",P16220(1)|P18848(1)
2,CPX-9,"bZIP transcription factor complex, ATF1-ATF4",ATF4-ATF1 transcription factor complex,9606,P18846(1)|P18848(1),ECO:0005544(biological system reconstruction e...,-,GO:0005634(nucleus)|GO:0000977(RNA polymerase ...,pubmed:23661758(see-also)|pubmed:28186491(see-...,Transcription factor complex which binds to a ...,Dimerization is mediated by the basic leucine-...,Heterodimer,-,-,-,-,-,"psi-mi:""MI:0469""(IntAct)",P18846(1)|P18848(1)
3,CPX-11,SMAD2 homotrimer,-,9606,Q15796(3),ECO:0000353(physical interaction evidence used...,intact:EBI-25772094,GO:0071142(homomeric SMAD protein complex),pubmed:16322555(see-also)|wwpdb:1khx(identity)...,"In the absence of Smad4, R-Smad phosphorylatio...",The R-Smad homotrimer is stabilized by the int...,Homotrimer,-,-,-,-,-,"psi-mi:""MI:0469""(IntAct)",Q15796(3)
4,CPX-12,SMAD3 homotrimer,-,9606,P84022(3),ECO:0005546(biological system reconstruction e...,-,GO:0071142(homomeric SMAD protein complex),pubmed:16322555(see-also)|complex portal:CPX-1...,"In the absence of Smad4, R-Smad phosphorylatio...",The R-Smad homotrimer is stabilized by the int...,Homotrimer,-,-,-,-,-,"psi-mi:""MI:0469""(IntAct)",P84022(3)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2341,CPX-3288,HSP90A-CDC37 chaperone complex,-,9606,P07900(2)|Q16543(2),ECO:0000353(physical interaction evidence used...,intact:EBI-9371424,GO:1990565(HSP90-CDC37 chaperone complex)|GO:0...,pubmed:19387550(see-also)|wwpdb:2k5b(identity)...,A protein kinase chaperone complex required fo...,CDC37 binds its target proteins via its N-term...,Heterotetramer,-,-,-,-,-,"psi-mi:""MI:0469""(IntAct)",P07900(2)|Q16543(2)
2342,CPX-3290,Interleukin-23 complex,IL-23 complex|p19p40 complex|IL23 complex|p19-...,9606,P29460-PRO_0000010930(1)|Q9NPF7-PRO_0000259488(1),ECO:0000353(physical interaction evidence used...,intact:EBI-9633865,GO:0006955(immune response)|GO:0032729(positiv...,pubmed:25516297(see-also)|reactome:R-HSA-44721...,Cytokine complex that activates and stimulates...,-,Heterodimer,-,Psoriasis [EFO:0000676]: a common genetically ...,-,-,-,"psi-mi:""MI:0469""(IntAct)",P29460-PRO_0000010930(1)|Q9NPF7-PRO_0000259488(1)
2343,CPX-3291,"SCF E3 ubiquitin ligase complex, FBXL3 variant",CRL1-FBXL3 complex|SCF-FBXL3 complex,9606,P62877(1)|P63208(1)|Q13616(1)|Q9UKT7(1),ECO:0005546(biological system reconstruction e...,-,GO:0042752(regulation of circadian rhythm)|GO:...,wwpdb:1LDJ(subset)|pubmed:17463251(see-also)|c...,E3 ubiquitin ligase which catalyzes the transf...,The cullin protein (CUL1) acts as an assembly ...,Heterotetramer,-,-,-,-,-,"psi-mi:""MI:2228""(ceitec)",P62877(1)|P63208(1)|Q13616(1)|Q9UKT7(1)
2344,CPX-3292,"SCF E3 ubiquitin ligase complex, FBXL2 variant",CRL1-FBXL2 complex|SCF-FBXL2 complex,9606,P62877(1)|P63208(1)|Q13616(1)|Q9UKC9(1),ECO:0005546(biological system reconstruction e...,-,GO:0050727(regulatio

In [10]:
# write a parser that takes each uniprot name in Expanded participant list and strips the (number) and separates by "|"
# translate each uniprot ID to the primary gene name (can use the idmapping table from uniprot)
# create final table that has duplicate protein complexes for each gene
# use the Gene_A and Gene_B columns in pandas dataframe from gene_similarity_gobp_value.pkl, for each gene put the complex IDs in two cols: Gene_A_complex and Gene_B_complex 
# write logic that says if complex columns are the same value, then create a new column and in that column put a '1', otherwise, put a '0'

In [11]:
idmapping_df = pd.read_csv('idmapping_2025_04_02.tsv', sep='\t')
idmapping_df

,From,Entry,Entry Name,Gene Names (primary),Gene Ontology (biological process)
0,A0A087X1C5,A0A087X1C5,CP2D7_HUMAN,CYP2D7,arachidonate metabolic process [GO:0019369]; x...
1,A0A0B4J2F0,A0A0B4J2F0,PIOS1_HUMAN,PIGBOS1,regulation of endoplasmic reticulum unfolded p...
2,A0A0B4J2F2,A0A0B4J2F2,SIK1B_HUMAN,NaN,NaN
3,A0A0C5B5G6,A0A0C5B5G6,MOTSC_HUMAN,MT-RNR1,activation of protein kinase activity [GO:0032...
4,A0A0K2S4Q6,A0A0K2S4Q6,CD3CH_HUMAN,CD300H,neutrophil chemotaxis [GO:0030593]; regulation...
...,...,...,...,...,...
20415,Q9UI54,Q9UI54,YT001_HUMAN,NaN,NaN
20416,Q9UI72,Q9UI72,YE014_HUMAN,NaN,NaN
20417,Q9Y3F1,Q9Y3F1,TA6P_HUMAN,NaN,NaN
20418,Q9Y6C7,Q9Y6C7,L3R2A_HUMAN,LINC00312,NaN


In [12]:
# Step 1: Clean and split UniProt IDs in the "Expanded participant list"
def parse_uniprot_ids(expanded_str):
    if pd.isna(expanded_str):
        return []
    # Remove stoichiometry in parentheses and split by '|'
    return [re.sub(r'\(\d+\)', '', item) for item in expanded_str.split('|')]

# Apply to the dataframe
human_complexes['UniProt_IDs'] = human_complexes['Expanded participant list'].apply(parse_uniprot_ids)


In [13]:
human_complexes

,#Complex ac,Recommended name,Aliases for complex,Taxonomy identifier,Identifiers (and stoichiometry) of molecules in complex,Evidence Code,Experimental evidence,Go Annotations,Cross references,Description,Complex properties,Complex assembly,Ligand,Disease,Agonist,Antagonist,Comment,Source,Expanded participant list,UniProt_IDs
0,CPX-1,SMAD2-SMAD3-SMAD4 complex,SMAD2/SMAD3/SMAD4 transcription factor complex,9606,P84022(1)|Q13485(1)|Q15796(1),ECO:0005547(biological system reconstruction e...,-,GO:0071144(heteromeric SMAD protein complex)|G...,reactome:R-HSA-9736938(identity)|reactome:R-HS...,A transcription factor complex which binds to ...,Preferential formation of the regulatory R-Sma...,Heterotrimer,-,-,-,-,-,"psi-mi:""MI:0469""(IntAct)",P84022(1)|Q13485(1)|Q15796(1),"[P84022, Q13485, Q15796]"
1,CPX-8,"bZIP transcription factor complex, ATF4-CREB1",-,9606,P16220(1)|P18848(1),ECO:0005544(biological system reconstruction e...,-,GO:0000977(RNA polymerase II transcription reg...,complex portal:CPX-8(complex-primary)|complex ...,Transcription factor complex that binds the cA...,-,Heterodimer,-,-,-,-,-,"psi-mi:""MI:0469""(IntAct)",P16220(1)|P18848(1),"[P16220, P18848]"
2,CPX-9,"bZIP transcription factor complex, ATF1-ATF4",ATF4-ATF1 transcription factor complex,9606,P18846(1)|P18848(1),ECO:0005544(biological system reconstruction e...,-,GO:0005634(nucleus)|GO:0000977(RNA polymerase ...,pubmed:23661758(see-also)|pubmed:28186491(see-...,Transcription factor complex which binds to a ...,Dimerization is mediated by the basic leucine-...,Heterodimer,-,-,-,-,-,"psi-mi:""MI:0469""(IntAct)",P18846(1)|P18848(1),"[P18846, P18848]"
3,CPX-11,SMAD2 homotrimer,-,9606,Q15796(3),ECO:0000353(physical interaction evidence used...,intact:EBI-25772094,GO:0071142(homomeric SMAD protein complex),pubmed:16322555(see-also)|wwpdb:1khx(identity)...,"In the absence of Smad4, R-Smad phosphorylatio...",The R-Smad homotrimer is stabilized by the int...,Homotrimer,-,-,-,-,-,"psi-mi:""MI:0469""(IntAct)",Q15796(3),[Q15796]
4,CPX-12,SMAD3 homotrimer,-,9606,P84022(3),ECO:0005546(biological system reconstruction e...,-,GO:0071142(homomeric SMAD protein complex),pubmed:16322555(see-also)|complex portal:CPX-1...,"In the absence of Smad4, R-Smad phosphorylatio...",The R-Smad homotrimer is stabilized by the int...,Homotrimer,-,-,-,-,-,"psi-mi:""MI:0469""(IntAct)",P84022(3),[P84022]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2341,CPX-3288,HSP90A-CDC37 chaperone complex,-,9606,P07900(2)|Q16543(2),ECO:0000353(physical interaction evidence used...,intact:EBI-9371424,GO:1990565(HSP90-CDC37 chaperone complex)|GO:0...,pubmed:19387550(see-also)|wwpdb:2k5b(identity)...,A protein kinase chaperone complex required fo...,CDC37 binds its target proteins via its N-term...,Heterotetramer,-,-,-,-,-,"psi-mi:""MI:0469""(IntAct)",P07900(2)|Q16543(2),"[P07900, Q16543]"
2342,CPX-3290,Interleukin-23 complex,IL-23 complex|p19p40 complex|IL23 complex|p19-...,9606,P29460-PRO_0000010930(1)|Q9NPF7-PRO_0000259488(1),ECO:0000353(physical interaction evidence used...,intact:EBI-9633865,GO:0006955(immune response)|GO:0032729(positiv...,pubmed:25516297(see-also)|reactome:R-HSA-44721...,Cytokine complex that activates and stimulates...,-,Heterodimer,-,Psoriasis [EFO:0000676]: a common genetically ...,-,-,-,"psi-mi:""MI:0469""(IntAct)",P29460-PRO_0000010930(1)|Q9NPF7-PRO_0000259488(1),"[P29460-PRO_0000010930, Q9NPF7-PRO_0000259488]"
2343,CPX-3291,"SCF E3 ubiquitin ligase complex, FBXL3 variant",CRL1-FBXL3 complex|SCF-FBXL3 complex,9606,P62877(1)|P63208(1)|Q13616(1)|Q9UKT7(1),ECO:0005546(biological system reconstruction e...,-,GO:0042752(regulation of circadian rhythm)|GO:...,wwpdb:1LDJ(subset)|pubmed:17463251(see-also)|c...,E3 ubiquitin ligase which catalyzes the transf...,The cullin protein (CUL1) acts as an assembly ...,Heterotetramer,-,-,-,-,-,"psi-mi:""MI:2228""(ceitec)",P62877(1)|P63208(1)|Q13616(1)|Q9UKT7(1),"[P62877, P63208, Q13616, Q9UKT7]"
2344,CPX-3292,"

In [15]:
# Step 2: Create a mapping dictionary from UniProt ID to gene name
uniprot_to_gene = idmapping_df.set_index('From')['Gene Names (primary)'].dropna().to_dict()

In [18]:
# Step 3: Explode complex-gene relationships into rows
rows = []
for _, row in human_complexes.iterrows():
    complex_id = row['#Complex ac']
    for uniprot_id in row['UniProt_IDs']:
        gene_name = uniprot_to_gene.get(uniprot_id)
        if gene_name:
            rows.append({'gene_name': gene_name, 'complex_id': complex_id})

gene_complex_df = pd.DataFrame(rows)

In [19]:
gene_complex_df

,gene_name,complex_id
0,SMAD3,CPX-1
1,SMAD4,CPX-1
2,SMAD2,CPX-1
3,CREB1,CPX-8
4,ATF4,CPX-8
...,...,...
8807,FBXL2,CPX-3292
8808,RBX1,CPX-3295
8809,SKP1,CPX-3295
8810,SKP2,CPX-3295


In [26]:
# Load gene similarity dataframe
similarity_df = pd.read_pickle("gene_similarity_gobp_value.pkl")

In [41]:
# Merge to add complex IDs
merged_df = similarity_df.merge(gene_complex_df.rename(columns={'gene_name': 'Gene_A', 'complex_id': 'Gene_A_complex'}),
                                on='Gene_A', how='left')

In [42]:
merged_df = merged_df.merge(gene_complex_df.rename(columns={'gene_name': 'Gene_B', 'complex_id': 'Gene_B_complex'}),
                            on='Gene_B', how='left')

# Compare complexes and create indicator column
merged_df['Same_Complex'] = (merged_df['Gene_A_complex'] == merged_df['Gene_B_complex']).astype(int)


In [48]:
merged_df.to_pickle('gene_pairs_complex_label.pkl')

In [44]:
merged_df['Same_Complex'].value_counts()

0    325205778
1        39275
Name: Same_Complex, dtype: int64

In [25]:
merged_df.dropna(subset='Gene_A_complex')

,Gene_A,Gene_B,Similarity,GOBP_value,Gene_A_complex,Gene_B_complex
52566,WASH2P,PI4KAP1,0.306697,0.0,CPX-1168,NaN
52567,WASH2P,PI4KAP1,0.306697,0.0,CPX-1173,NaN
52568,WASH2P,ZNF833P,0.261358,0.0,CPX-1168,NaN
52569,WASH2P,ZNF833P,0.261358,0.0,CPX-1173,NaN
52570,WASH2P,FCGR2C,0.145629,0.0,CPX-1168,NaN
...,...,...,...,...,...,...
325106373,IL4R,SIX5,0.108147,0.0,CPX-624,NaN
325106374,IL4R,SIX5,0.108147,0.0,CPX-844,NaN
325106375,IL4R,NARS2,0.069624,0.0,CPX-8834,NaN
325106376,IL4R,NARS2,0.069624,0.0,CPX-624,NaN


In [24]:
similarity_df.shape

(200794875, 4)

In [30]:
ma = merged_df.dropna()

In [31]:
ma

,Gene_A,Gene_B,Similarity,GOBP_value,Gene_A_complex
41518,WASH2P,PI4KAP1,0.306697,0.0,CPX-1168
41519,WASH2P,PI4KAP1,0.306697,0.0,CPX-1173
41520,WASH2P,ZNF833P,0.261358,0.0,CPX-1168
41521,WASH2P,ZNF833P,0.261358,0.0,CPX-1173
41522,WASH2P,FCGR2C,0.145629,0.0,CPX-1168
...,...,...,...,...,...
256388304,IL4R,SIX5,0.108147,0.0,CPX-624
256388305,IL4R,SIX5,0.108147,0.0,CPX-844
256388306,IL4R,NARS2,0.069624,0.0,CPX-8834
256388307,IL4R,NARS2,0.069624,0.0,CPX-624


In [33]:
mb = ma.merge(gene_complex_df.rename(columns={'gene_name': 'Gene_B', 'complex_id': 'Gene_B_complex'}),
                            on='Gene_B', how='inner')

In [34]:
mb

,Gene_A,Gene_B,Similarity,GOBP_value,Gene_A_complex,Gene_B_complex
0,WASH2P,PSPC1,-0.041077,0.0,CPX-1168,CPX-889
1,WASH2P,PSPC1,-0.041077,0.0,CPX-1168,CPX-7764
2,WASH2P,PSPC1,-0.041077,0.0,CPX-1168,CPX-7783
3,WASH2P,PSPC1,-0.041077,0.0,CPX-1173,CPX-889
4,WASH2P,PSPC1,-0.041077,0.0,CPX-1173,CPX-7764
...,...,...,...,...,...,...
38611159,FBXO3,RBX1,-0.007389,1.0,CPX-7881,CPX-2859
38611160,FBXO3,RBX1,-0.007389,1.0,CPX-7881,CPX-2873
38611161,FBXO3,RBX1,-0.007389,1.0,CPX-7881,CPX-3291
38611162,FBXO3,RBX1,-0.007389,1.0,CPX-7881,CPX-3292


In [35]:
mb['Same_Complex'] = (mb['Gene_A_complex'] == mb['Gene_B_complex']).astype(int)


In [36]:
mb

,Gene_A,Gene_B,Similarity,GOBP_value,Gene_A_complex,Gene_B_complex,Same_Complex
0,WASH2P,PSPC1,-0.041077,0.0,CPX-1168,CPX-889,0
1,WASH2P,PSPC1,-0.041077,0.0,CPX-1168,CPX-7764,0
2,WASH2P,PSPC1,-0.041077,0.0,CPX-1168,CPX-7783,0
3,WASH2P,PSPC1,-0.041077,0.0,CPX-1173,CPX-889,0
4,WASH2P,PSPC1,-0.041077,0.0,CPX-1173,CPX-7764,0
...,...,...,...,...,...,...,...
38611159,FBXO3,RBX1,-0.007389,1.0,CPX-7881,CPX-2859,0
38611160,FBXO3,RBX1,-0.007389,1.0,CPX-7881,CPX-2873,0
38611161,FBXO3,RBX1,-0.007389,1.0,CPX-7881,CPX-3291,0
38611162,FBXO3,RBX1,-0.007389,1.0,CPX-7881,CPX-3292,0


In [37]:
mb['Same_Complex'].value_counts()

0    38571889
1       39275
Name: Same_Complex, dtype: int64

In [38]:
mb

,Gene_A,Gene_B,Similarity,GOBP_value,Gene_A_complex,Gene_B_complex,Same_Complex
0,WASH2P,PSPC1,-0.041077,0.0,CPX-1168,CPX-889,0
1,WASH2P,PSPC1,-0.041077,0.0,CPX-1168,CPX-7764,0
2,WASH2P,PSPC1,-0.041077,0.0,CPX-1168,CPX-7783,0
3,WASH2P,PSPC1,-0.041077,0.0,CPX-1173,CPX-889,0
4,WASH2P,PSPC1,-0.041077,0.0,CPX-1173,CPX-7764,0
...,...,...,...,...,...,...,...
38611159,FBXO3,RBX1,-0.007389,1.0,CPX-7881,CPX-2859,0
38611160,FBXO3,RBX1,-0.007389,1.0,CPX-7881,CPX-2873,0
38611161,FBXO3,RBX1,-0.007389,1.0,CPX-7881,CPX-3291,0
38611162,FBXO3,RBX1,-0.007389,1.0,CPX-7881,CPX-3292,0


In [39]:
similarity_df

,Gene_A,Gene_B,Similarity,GOBP_value
4,ST13P4,WASH2P,0.357125,0.0
9,ST13P4,FAM90A12P,0.106240,0.0
13,ST13P4,PI4KAP1,0.160441,0.0
17,ST13P4,SNX29P2,0.211458,0.0
19,ST13P4,ZNF818P,0.100117,0.0
...,...,...,...,...
403312697,RBMXP1,PPT1,-0.122568,0.0
403312698,RBMXP1,PLD2,0.069859,0.0
403312699,RBMXP1,EIF4EBP1,-0.037521,0.0
403312700,RBMXP1,TINAG,0.135589,0.0
